In [ ]:
import os
import torch
import sys
from PIL import Image
import numpy as np
import pandas as pd
import ultralytics
import cv2
from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/drive/')

In [ ]:
%cd /content/drive/My Drive/DL/eKYC/

!pip install -r requirements.txt 

!pip install --quiet vietocr==0.3.5

!pip install flask-ngrok
!pip install flask==0.12.2
!pip install pyngrok

# restart runtime
os.kill(os.getpid(), 9)

In [ ]:
# Import
py_file_location = "/content/drive/My Drive/DL/eKYC"
sys.path.append(os.path.abspath(py_file_location))
%cd /content/drive/My Drive/DL/eKYC
import crop_image
import detect_word

In [ ]:
%cd /content/drive/My Drive/eKYC/src/models
#Load model
#Load model cropper
model_crop = YOLO('./detect_corners.pt')

#Load model detecter
model_detect = YOLO('./extract_info')

#Load model OCR (reader)
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
config = Cfg.load_config_from_name('vgg_transformer')
config['weights'] = 'transformerocr.pth'
config['cnn']['pretrained']=False
config['device'] = 'cpu'
config['predictor']['beamsearch']=False

detector = Predictor(config)


In [ ]:
# restart runtime
os.kill(os.getpid(), 9)

In [ ]:
%cd /content/drive/My Drive/DL/eKYC

from flask import Flask, flash, request, redirect, url_for, render_template
import urllib.request
import os
import threading
from pyngrok import ngrok

from werkzeug.utils import secure_filename

# # run_with_ngrok
# from flask_ngrok import run_with_ngrok 
# app = Flask(__name__)
# run_with_ngrok(app)

os.environ["FLASK_ENV"] = "development"
app = Flask(__name__)
port = 5000
# auth_token  -  https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("[YOUR_TOKEN]")
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url

print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))
# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url  


UPLOAD_FOLDER = 'static/uploads/'
 
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
 
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])
 
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
     
 
@app.route('/')
def hometown():
    return render_template('index.html')

#Get image and OCR
@app.route('/', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        flash('No file part')
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        flash('No image selected for uploading')
        return redirect(request.url)
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        #print('upload_image filename: ' + filename)

#OCR
        image = cv2.imread(os.path.join('static/uploads',filename))
        # cv2_imshow(image)
        # predict (Cropper)
        results_crop = model_crop(image, size=640)
        # crop
        try:
          for j in range(len(results_crop.xyxy)):
            img_crop = crop_image.CropImg(results_crop.xyxy[j],image)
        except BaseException:
          return render_template('index.html', filename=filename, id ="None", name ="None", dob ="None", hometown ="None")
        # Detector
        try:
          results_detect = model_detect(img_crop, size=640)
        except BaseException:
          return render_template('index.html', filename=filename, id ="None", name ="None", dob ="None", hometown ="None")
        #OCR
        try:
          for j in range(len(results_detect.xyxy)):
            dic = detect_word.OCR(results_detect.xyxy[j],results_detect.names,img_crop, detector)
        except BaseException:
          return render_template('index.html', filename=filename, id ="None", name ="None", dob ="None", hometown ="None")

        # print(dic)
        #miss infor
        if "id" not in dic:
          dic['id'] = 'None'
        if "name" not in dic:
          dic['name'] = 'None'
        if "dob" not in dic:
          dic['dob'] = 'None'
        if "hometown" not in dic:
          dic['hometown'] = 'None'
 
        print(dic)
        return render_template('index.html', filename=filename, id =dic["id"], name =dic["name"], dob =dic["dob"], hometown =dic["hometown"])
    
    else:
        flash('Allowed image types are - png, jpg, jpeg, gif')
        return redirect(request.url)
 

# Load image
@app.route('/display/<filename>')
def display_image(filename):
    #print('display_image filename: ' + filename)
    return redirect(url_for('static', filename='uploads/' + filename), code=301)

# # run_with_ngrok 
# if __name__ == "__main__":
#     app.run()

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()